In [1]:
import datetime
import os
import pathlib
import warnings

import load_ribasim  # noqa: F401
import pandas as pd
import ribasim
import ribasim.nodes

import peilbeheerst_model.ribasim_parametrization as ribasim_param
from peilbeheerst_model.add_storage_basins import AddStorageBasins
from peilbeheerst_model.controle_output import Control
from peilbeheerst_model.ribasim_feedback_processor import RibasimFeedbackProcessor
from peilbeheerst_model.assign_authorities import AssignAuthorities

from ribasim import Node
from ribasim.nodes import pump, level_boundary, tabulated_rating_curve
from shapely import Point

%reload_ext autoreload
%autoreload 2
warnings.filterwarnings("ignore")

## Define variables and model

#### Set Config

In [2]:
waterschap = "Rivierenland"
work_dir = pathlib.Path(f"../../../../../Ribasim_updated_models/{waterschap}/modellen/{waterschap}_parametrized")
ribasim_gpkg = work_dir.joinpath("database.gpkg")
path_ribasim_toml = work_dir.joinpath("ribasim.toml")
output_dir = work_dir.joinpath("results")

# Basin area percentage
regular_percentage = 10
boezem_percentage = 90
unknown_streefpeil = (
    0.00012345  # we need a streefpeil to create the profiles, Q(h)-relations, and af- and aanslag peil for pumps
)

# Forcing settings
start_time = "2024-01-01"
timestep_size = "d"
timesteps = 2
delta_crest_level = 0.1  # delta waterlevel of boezem compared to streefpeil till no water can flow through an outlet

default_level = 0.60  # default LevelBoundary level, +- level at Kinderdijk

## Process the feedback form

In [3]:
name = "Ron Bruijns (HKV)"
versie = "2024_12_0"

feedback_excel = pathlib.Path(f"../../../../../Ribasim_feedback/V1_formulieren/feedback_formulier_{waterschap}.xlsx")
feedback_excel_processed = (
    f"../../../../..//Ribasim_feedback/V1_formulieren_verwerkt/feedback_formulier_{waterschap}_JA_processed.xlsx"
)

ribasim_toml = f"../../../../../Ribasim_base_models/{waterschap}_boezemmodel_{versie}/ribasim.toml"
output_folder = work_dir  # f"../../../../../Ribasim_updated_models/{waterschap}"

processor = RibasimFeedbackProcessor(
    name, waterschap, versie, feedback_excel, ribasim_toml, output_folder, feedback_excel_processed, use_validation=True
)
processor.run()

Swapped edge direction between Node A: 242 and Node B: 792
Swapped edge direction between Node A: 792 and Node B: 195
Swapped edge direction between Node A: 241 and Node B: 777
Swapped edge direction between Node A: 777 and Node B: 191
Swapped edge direction between Node A: 940 and Node B: 716
Swapped edge direction between Node A: 716 and Node B: 151
Swapped edge direction between Node A: 206 and Node B: 825
Swapped edge direction between Node A: 825 and Node B: 103
Swapped edge direction between Node A: 154 and Node B: 715
Swapped edge direction between Node A: 715 and Node B: 939
Swapped edge direction between Node A: 233 and Node B: 476
Swapped edge direction between Node A: 476 and Node B: 70
Swapped edge direction between Node A: 293 and Node B: 228
Swapped edge direction between Node A: 18 and Node B: 293
Swapped edge direction between Node A: 1 and Node B: 659
Swapped edge direction between Node A: 659 and Node B: 145
Swapped edge direction between Node A: 1 and Node B: 660
Swa

#### Load model

In [4]:
# Load Ribasim model
with warnings.catch_warnings():
    warnings.simplefilter(action="ignore", category=FutureWarning)
    ribasim_model = ribasim.Model(filepath=path_ribasim_toml)

# Parameterization

## Nodes

### Basin (characteristics)

In [5]:
ribasim_param.validate_basin_area(ribasim_model)

All basins are larger than 100 m²


In [6]:
# remove the basins of above in the feedback form

## Model specific tweaks

In [7]:
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

In [8]:
# change unknown streefpeilen to a default streefpeil
ribasim_model.basin.area.df.loc[
    ribasim_model.basin.area.df["meta_streefpeil"] == "Onbekend streefpeil", "meta_streefpeil"
] = str(unknown_streefpeil)
ribasim_model.basin.area.df.loc[ribasim_model.basin.area.df["meta_streefpeil"] == -9.999, "meta_streefpeil"] = str(
    unknown_streefpeil
)

In [9]:
assert not pd.isnull(ribasim_model.basin.area.df.meta_streefpeil).any()

In [10]:
#The spoelbak of Kinderdijk is not able to discharge water due to a lower default streefpeil level than the Lek. Change it manually. 36 discharges on 40, so make 36 larger
ribasim_model.basin.state.df.loc[ribasim_model.basin.state.df.node_id == 36, 'level'] = 4
ribasim_model.basin.state.df.loc[ribasim_model.basin.state.df.node_id == 40, 'level'] = 3

ribasim_model.basin.area.df.loc[ribasim_model.basin.area.df.node_id == 36, 'meta_streefpeil'] = str(4)
ribasim_model.basin.area.df.loc[ribasim_model.basin.area.df.node_id == 40, 'meta_streefpeil'] = str(3)


#add levelboundary and a pump
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(136538, 422962)), [level_boundary.Static(level=[default_level])])

pump_node = ribasim_model.pump.add(Node(new_node_id+1, Point(136574, 422965)), [pump.Static(flow_rate=[0.1])])

#add static information
ribasim_model.pump.static.df.loc[ribasim_model.pump.static.df.node_id == new_node_id + 1, 'meta_func_aanvoer'] = 1 #its a wateraanvoer pump according FW
ribasim_model.edge.add(ribasim_model.basin[154], pump_node)
ribasim_model.edge.add(pump_node, level_boundary_node)

#add gemaal and LB at Pannerlingen
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(198612,434208)), [level_boundary.Static(level=[default_level])])

pump_node = ribasim_model.pump.add(Node(new_node_id+1, Point(198568,434184)), [pump.Static(flow_rate=[0.1])])

ribasim_model.edge.add(level_boundary_node, pump_node)
ribasim_model.edge.add(pump_node, ribasim_model.basin[115])


#add a TRC and LB near Groesbeek to Germany
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(196158,421051)), [level_boundary.Static(level=[default_level])])

tabulated_rating_curve_node = ribasim_model.tabulated_rating_curve.add(Node(new_node_id+1, Point(196147,421056)),
                                                                      [tabulated_rating_curve.Static(
                                                                        level=[0.0, 0.1234],
                                                                        flow_rate=[0.0, 0.1234])])
ribasim_model.edge.add(ribasim_model.basin[237], tabulated_rating_curve_node)
ribasim_model.edge.add(tabulated_rating_curve_node, level_boundary_node)


#add a TRC and LB near Groesbeek to Germany
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(103311,433732)), [level_boundary.Static(level=[default_level])])

tabulated_rating_curve_node = ribasim_model.tabulated_rating_curve.add(Node(new_node_id+1, Point(103315,433716)),
                                                                      [tabulated_rating_curve.Static(
                                                                        level=[0.0, 0.1234],
                                                                        flow_rate=[0.0, 0.1234])])
ribasim_model.edge.add(ribasim_model.basin[36], tabulated_rating_curve_node)
ribasim_model.edge.add(tabulated_rating_curve_node, level_boundary_node)

In [11]:
#add gemaal and LB at downstream Linge (Hardinxveld)
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

pump_node = ribasim_model.pump.add(Node(new_node_id+1, Point(118539.25,425972.46)), [pump.Static(flow_rate=[20])])
level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(118530.65,425964)), [level_boundary.Static(level=[default_level])])

ribasim_model.edge.add(ribasim_model.basin[1], pump_node)
ribasim_model.edge.add(pump_node, level_boundary_node)


#add gemaal and LB at downstream Linge (Hardinxveld)
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

pump_node = ribasim_model.pump.add(Node(new_node_id+1, Point(118561.5,425973.0)), [pump.Static(flow_rate=[20])])
level_boundary_node = ribasim_model.level_boundary.add(Node(new_node_id, Point(118548,425951)), [level_boundary.Static(level=[default_level])])

ribasim_model.edge.add(ribasim_model.basin[1], pump_node)
ribasim_model.edge.add(pump_node, level_boundary_node)

In [12]:
ribasim_model.level_boundary.node.df.meta_node_id = ribasim_model.level_boundary.node.df.index
ribasim_model.tabulated_rating_curve.node.df.meta_node_id = ribasim_model.tabulated_rating_curve.node.df.index
ribasim_model.pump.node.df.meta_node_id = ribasim_model.pump.node.df.index



## Implement standard profile and a storage basin

In [13]:
# Insert standard profiles to each basin. These are [depth_profiles] meter deep, defined from the streefpeil
ribasim_param.insert_standard_profile(
    ribasim_model,
    unknown_streefpeil=unknown_streefpeil,
    regular_percentage=regular_percentage,
    boezem_percentage=boezem_percentage,
    depth_profile=2,
)

In [14]:
add_storage_basins = AddStorageBasins(
    ribasim_model=ribasim_model, exclude_hoofdwater=True, additional_basins_to_exclude=[]
)

add_storage_basins.create_bergende_basins()


### Basin (forcing)

In [15]:
# Set static forcing
forcing_dict = {
    "precipitation": ribasim_param.convert_mm_day_to_m_sec(10*2),
    "potential_evaporation": ribasim_param.convert_mm_day_to_m_sec(0),
    "drainage": ribasim_param.convert_mm_day_to_m_sec(0),
    "infiltration": ribasim_param.convert_mm_day_to_m_sec(0),
    # 'urban_runoff':          ribasim_param.convert_mm_day_to_m_sec(0),
}

ribasim_param.set_static_forcing(timesteps, timestep_size, start_time, forcing_dict, ribasim_model)

### Pumps

In [16]:
# Set pump capacity for each pump
ribasim_model.pump.static.df["flow_rate"] = 0.16667  # 10 kuub per minuut

### Convert all boundary nodes to LevelBoundaries

In [17]:
ribasim_param.Terminals_to_LevelBoundaries(ribasim_model=ribasim_model, default_level=default_level)  # clean
ribasim_param.FlowBoundaries_to_LevelBoundaries(ribasim_model=ribasim_model, default_level=default_level)

### Add Outlet

In [18]:
ribasim_param.add_outlets(ribasim_model, delta_crest_level=0.10)

## Add control, based on the meta_categorie

In [19]:
ribasim_param.identify_node_meta_categorie(ribasim_model)

In [20]:
ribasim_param.find_upstream_downstream_target_levels(ribasim_model, node="outlet")
ribasim_param.find_upstream_downstream_target_levels(ribasim_model, node="pump")


In [21]:
#change the control of the outlet at Kinderdijk
ribasim_model.outlet.static.df.loc[ribasim_model.outlet.static.df.node_id == 355, 'min_upstream_level'] = 2
ribasim_model.pump.static.df.loc[ribasim_model.pump.static.df.node_id == 280, 'meta_categorie'] = 'Inlaat boezem, afvoer gemaal'


In [22]:
# ribasim_param.add_discrete_control(ribasim_model, waterschap, default_level)

In [23]:
ribasim_param.determine_min_upstream_max_downstream_levels(ribasim_model, waterschap)

Warning! Some pumps do not have a flow rate yet. Dummy value of 0.1234 m3/s has been taken.


### Manning Resistance

In [24]:
# there is a MR without geometry and without edges for some reason
ribasim_model.manning_resistance.node.df = ribasim_model.manning_resistance.node.df.dropna(subset="geometry")

In [25]:
#lower the difference in waterlevel for each manning node
ribasim_model.manning_resistance.static.df.length = 10
ribasim_model.manning_resistance.static.df.manning_n = 0.01

## Last formating of the tables

In [26]:
# only retain node_id's which are present in the .node table
ribasim_param.clean_tables(ribasim_model, waterschap)

# Set numerical settings

In [27]:
ribasim_model.use_validation = True

In [28]:
# Write model output
# ribasim_param.index_reset(ribasim_model)
ribasim_model.starttime = datetime.datetime(2024, 1, 1)
ribasim_model.endtime = datetime.datetime(2024, 4, 1)
ribasim_model.solver.saveat = 3600*24
ribasim_param.write_ribasim_model_Zdrive(ribasim_model, path_ribasim_toml)

## Run Model

## Iterate over tabulated rating curves

In [29]:
ribasim_param.tqdm_subprocess(["ribasim", path_ribasim_toml], 
                              print_other=False, 
                              suffix="init")

Simulating init:   0%|          | 0/100 [00:00<?, ?it/s]

In [30]:
controle_output = Control(work_dir=work_dir)
indicators = controle_output.run_all()

# Write model

In [31]:
# control_dict = Control(work_dir = work_dir).run_all()
ribasim_param.write_ribasim_model_GoodCloud(
    ribasim_model=ribasim_model,
    path_ribasim_toml=path_ribasim_toml,
    waterschap=waterschap,
    modeltype="boezemmodel",
    include_results=True,
)

../../../../../Ribasim_networks/Waterschappen/Rivierenland/modellen/Rivierenland_parametrized/results
The model of waterboard Rivierenland has been uploaded to the goodcloud in the directory of boezemmodel!
